In [1]:
import PIL.Image

import numpy as np
import pkg_resources
import pickle
import matplotlib.pyplot as plt
from pathlib import Path
from flygym.envs.nmf_mujoco import NeuroMechFlyMuJoCo, MuJoCoParameters
from tqdm import trange
from flygym.util.config import all_leg_dofs
from flygym.state import stretched_pose

import cv2


In [2]:
# Initialize the simulation
adhesion = True
run_time = 1
sim_params = MuJoCoParameters(timestep=1e-4, render_mode="saved", render_playspeed=0.05, draw_joints=True)
nmf = NeuroMechFlyMuJoCo(
    sim_params=sim_params,
    init_pose=stretched_pose,
    actuated_joints=all_leg_dofs,
)

num_steps_base = int(run_time / nmf.timestep)

In [3]:
# Load recorded data
data_path = Path(pkg_resources.resource_filename('flygym', 'data'))
with open(data_path / 'behavior' / 'single_steps.pkl', 'rb') as f:
    data = pickle.load(f)

In [4]:
# Interpolate 5x
step_duration = len(data['joint_LFCoxa'])
interp_step_duration = int(step_duration * data['meta']['timestep'] / nmf.timestep)
data_block = np.zeros((len(nmf.actuated_joints), interp_step_duration))
measure_t = np.arange(step_duration) * data['meta']['timestep']
interp_t = np.arange(interp_step_duration) * nmf.timestep
for i, joint in enumerate(nmf.actuated_joints):
    data_block[i, :] = np.interp(interp_t, measure_t, data[joint])

In [5]:
n_stabilization_step = 2000

obs, _ = nmf.reset()

legs = ["RF", "LF", "RM", "LM", "RH", "LH"]
match_joints_to_leg = np.array([[i for i, joint in enumerate(nmf.actuated_joints) if leg in joint] for leg in legs])
obs_list = []

for i, l in enumerate(legs):
    if i == 0:
        for j in range(n_stabilization_step):
            joint_angles = data_block[:, 0]
            action = {"joints": joint_angles, "adhesion":np.zeros(6)}
            obs, _, _, _, _ = nmf.step(action)
            obs_list.append(obs)
            render_status = nmf.render()
        
    for j in range(interp_step_duration):
        joint_angles = data_block[:, 0]
        joint_angles[match_joints_to_leg[i]] = data_block[match_joints_to_leg[i], j]
        action = {"joints": joint_angles, "adhesion":np.zeros(6)}
        obs, _, _, _, _ = nmf.step(action)
        obs_list.append(obs)
        render_status = nmf.render()

In [6]:
video_path = Path(f"Single_step/single_step.mp4")
nmf.save_video(video_path, 0)

from ipywidgets import Video
Video.from_file(video_path)

Video(value=b'\x00\x00\x00 ftypisom\x00\x00\x02\x00isomiso2avc1mp41\x00\x00\x00\x08free...')